In [1]:
import pandas as pd
import random
import sqlalchemy
import scipy.stats as stats
import data
import importlib
import numpy as np

In [2]:
importlib.reload(data)

<module 'data' from 'C:\\Users\\amans\\Python\\monte-carlo-simulator\\data.py'>

In [3]:
engine = sqlalchemy.create_engine('sqlite:///wabonmerged.sqlite3')
measures = pd.read_sql('SELECT * FROM Measures;', engine)
matches = pd.read_sql('SELECT * FROM Matches;', engine)

In [4]:
last_match_for_averages = 'qm30'
matches_for_averages = data.filter_matches(matches, last_match=last_match_for_averages)
measures_for_averages = data.filter_measures(measures, last_match=last_match_for_averages)
matches_to_simulate = data.filter_matches(matches, first_match = 'qm31')

In [5]:
counts = data.get_counts(measures_for_averages)
taxi = data.get_taxi_sums(measures_for_averages)
hangar_totals = data.get_hangar_totals(measures_for_averages)

matches_to_simulate_data = data.get_matches(data.get_averages(taxi, counts, hangar_totals, matches_for_averages), matches_to_simulate)
#move hangar end level 0 column to the end for multinomial distribution purposes (see numpy documentation)
matches_to_simulate_data = matches_to_simulate_data.iloc[:, [i for i in range(9)]+[i for i in range(10,14)]+[9]]

In [25]:
scores=matches_to_simulate_data.copy() #0.0633995532989502 seconds per thousand iterations
scores.taxi = scores.taxi.apply(data.taxi)#1 second per thousand iterations # 1.19270038605 per thousand iterations
for column in ['auto_upper', 'auto_lower', 'tele_upper', 'tele_lower', 'endgame_penalty_count']: #2.14568018913 per thousand iterations
    scores[column]=stats.poisson.rvs(mu=scores[column])
for num in range(len(scores)):
    scores.iloc[num, -5:] = np.random.multinomial(1, scores.iloc[num, -5:])

In [26]:
cargo = (scores.loc[:, ['match','alliance','auto_upper', 'auto_lower', 'tele_upper', 'tele_lower']]
               .groupby(by=['match', 'alliance'])
               .sum()
               .assign(auto=lambda df: df.auto_lower+df.auto_upper,
                       tele = lambda df: df.tele_upper + df.tele_lower)
               .drop(['auto_lower', 'auto_upper', 'tele_upper', 'tele_lower'], axis = 1)
               .assign(total = lambda df: df.auto+df.tele)
               .assign(cargo_rp=lambda df: (df.total>=20)|((df.auto>=5)&(df.total>=18))))

In [28]:
scores_matrix = scores.iloc[:, 3:].to_numpy()
values_matrix = np.matrix('2; 4; -4; 1; 2; 5; 4; 6; 10; 15; 0')
points_matrix = np.dot(scores_matrix, values_matrix)

hangar_scores_matrix = scores.iloc[:, -5:].to_numpy()
hangar_values_matrix = np.matrix('4; 6; 10; 15; 0')
hangar_points_matrix = np.dot(hangar_scores_matrix, hangar_values_matrix)

scores = scores.assign(points = points_matrix, hangar_points = hangar_points_matrix)

In [38]:
rps = (scores.groupby(by=['match', 'alliance']).sum()
            .iloc[:, -2:]
            .assign(hangar_rp = lambda df: df.hangar_points>=16)
            .join(cargo))

In [45]:
rps

points  hangar_points  hangar_rp  auto  tele  total  cargo_rp
match alliance                                                               
qm31  blue        76.0           25.0       True     6    14     20      True
      red         53.0           15.0      False     3     9     12     False
qm32  blue        52.0            0.0      False     2    19     21      True
      red         53.0            6.0      False     6    12     18      True
qm33  blue        41.0            6.0      False     3     8     11     False
...                ...            ...        ...   ...   ...    ...       ...
qm60  red        107.0           25.0       True     3    30     33      True
qm61  blue        63.0           21.0       True     5     8     13     False
      red         25.0            4.0      False     2     5      7     False
qm62  blue        11.0            6.0      False     0     0      0     False
      red         16.0            4.0      False     0     2      2     False

[64 rows x 7 columns]

In [69]:
blue = (scores.groupby(by=['alliance','match'])
        .sum().loc['blue']
        .assign(win = scores.groupby(by=['alliance','match'])
                            .sum()
                            .loc['blue'].points>
                      scores.groupby(by=['alliance','match'])
                            .sum()
                            .loc['red'].points)
        .assign(alliance='blue')
        .reset_index()
        .set_index(['match', 'alliance'], drop=True))

In [73]:
blue.loc[:, 'win'] * 2

match  alliance
qm31   blue        2
qm32   blue        0
qm33   blue        0
qm34   blue        0
qm35   blue        0
qm36   blue        2
qm37   blue        0
qm38   blue        0
qm39   blue        0
qm40   blue        2
qm41   blue        2
qm42   blue        0
qm43   blue        0
qm44   blue        0
qm45   blue        2
qm46   blue        2
qm47   blue        2
qm48   blue        2
qm49   blue        0
qm50   blue        2
qm51   blue        0
qm52   blue        2
qm53   blue        2
qm54   blue        0
qm55   blue        2
qm56   blue        2
qm57   blue        0
qm58   blue        2
qm59   blue        2
qm60   blue        0
qm61   blue        2
qm62   blue        0
Name: win, dtype: int32